<a href="https://colab.research.google.com/github/manugupta-or/Coursera_aiscm/blob/main/Rajsthan_Logistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing necessary libraries
!pip install ortools
import numpy as np
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Storing data
dist = pd.read_excel('Rajasthan_logistics_data.xlsx')
#keep Rajsthan logistics data file in the same folder as 
dist = dist[1:len(dist)][:].iloc[:,1:8]

dist.columns = dist.iloc[0]
dist = dist.iloc[1:7,:]
dist = dist.set_index(dist.iloc[:,0],drop=True).iloc[:,1:7]
print(dist)

1         Udaipur Ajmer Bikaner Jaipur Jaisalmer Jodhpur
nan                                                     
Udaipur         0   265     501    394       490     251
Ajmer         265     0     284    135       484     206
Bikaner       501   284       0    342       331     252
Jaipur        394   135     342      0       558     339
Jaisalmer     490   484     331    558         0     283
Jodhpur       251   206     252    339       283       0


In [4]:
#creating all possible subsets
def subsets(numbers):
    if numbers == []:
        return [[]]
    x = subsets(numbers[1:])
    return x + [[numbers[0]] + y for y in x]
 
print(subsets([x for x  in range(len(dist))]))


[[], [5], [4], [4, 5], [3], [3, 5], [3, 4], [3, 4, 5], [2], [2, 5], [2, 4], [2, 4, 5], [2, 3], [2, 3, 5], [2, 3, 4], [2, 3, 4, 5], [1], [1, 5], [1, 4], [1, 4, 5], [1, 3], [1, 3, 5], [1, 3, 4], [1, 3, 4, 5], [1, 2], [1, 2, 5], [1, 2, 4], [1, 2, 4, 5], [1, 2, 3], [1, 2, 3, 5], [1, 2, 3, 4], [1, 2, 3, 4, 5], [0], [0, 5], [0, 4], [0, 4, 5], [0, 3], [0, 3, 5], [0, 3, 4], [0, 3, 4, 5], [0, 2], [0, 2, 5], [0, 2, 4], [0, 2, 4, 5], [0, 2, 3], [0, 2, 3, 5], [0, 2, 3, 4], [0, 2, 3, 4, 5], [0, 1], [0, 1, 5], [0, 1, 4], [0, 1, 4, 5], [0, 1, 3], [0, 1, 3, 5], [0, 1, 3, 4], [0, 1, 3, 4, 5], [0, 1, 2], [0, 1, 2, 5], [0, 1, 2, 4], [0, 1, 2, 4, 5], [0, 1, 2, 3], [0, 1, 2, 3, 5], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4, 5]]


In [6]:
from ortools.linear_solver import pywraplp
#solver = pywraplp.Solver.CreateSolver('SCIP')

#from ortools.sat.python import cp_model
#Initializing the Model.
model = pywraplp.Solver.CreateSolver('CBC')
#Decision Variable X[i,j] - 1 if availing route from city i to city j, 0 - otherwise
x=[[model.IntVar(0,1,'Binary Flag- %d%d' %(i,j)) for j in range(len(dist))] for i in range(len(dist))]

#Constraint of outward routees from the city is 1.
for i in range(len(dist)):
    row_sum=0
    for j in range(len(dist)):
        if(i!=j):
            row_sum=row_sum+x[i][j]   
    model.Add(row_sum==1)

#Constraint of inward routees to the city is 1.
for j in range(len(dist)):
    col_sum=0
    for i in range(len(dist)):
        if(i!=j):
            col_sum=col_sum+x[i][j]
        
    model.Add(col_sum==1)
#Subtour Elimination Constraint.
col_ind={}
#Getting all possible subsets.
for item in subsets([x for x  in range(len(dist))]):
    col_ind=[x for x  in range(len(dist))]
    #Considering subsets having more than one elements
    if(len(item)>=1):
        #Creating a set with elements no in the subset
        for val in item:
            col_ind.remove(val)
        #Adding the constraint that outward routes from the given set is atleast 1.
        if(len(col_ind)>=1):
            model.Add(sum([sum([x[i][j] for j in col_ind]) for i in item])>=1)
            
#Minimizing the objective for the distance travelled in the tour.
model.Minimize(sum([sum([x[i][j]*dist.iloc[i,j] for j in range(len(dist))]) for i in range(len(dist))]))

#solver = cp_model.CpSolver()

status = model.Solve()

print(status)
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
   # print('Status =', model.StatusName(status))
    print('Distance travelled =', model.Objective().Value())

    route=dist.columns[0]
    row = 0
    #Printing the optimum route.
    print("Optimum route:")
    for count in range(len(dist)):
        for j in range(len(dist)):
            if(x[row][j].solution_value()==1):
                route = route + "->" + dist.columns[j]
                row=j
                break
    print(route)

else:
    print('No solution found.')

0
Distance travelled = 1607.0
Optimum route:
Udaipur->Jodhpur->Jaisalmer->Bikaner->Jaipur->Ajmer->Udaipur


Q2. Security guard problem

In [ ]:

from ortools.linear_solver import pywraplp
def main():
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('CBC')#you can also use SCIP solver
    
    #Decision Variables
    x = [solver.IntVar(0, 100, 'New Joiners for shift %d'%i) for i in range(6)]
    
    #Constraints - Assumng availability of guards is cyclic across days
    solver.Add(x[4] + x[0] >= 5)
    solver.Add(x[5] + x[1] >= 12)
    solver.Add(x[0] + x[2] >= 8)
    solver.Add(x[1] + x[3] >= 11)
    solver.Add(x[2] + x[4] >= 8)
    solver.Add(x[3] + x[5] >= 19)
    
    #Objective
    solver.Minimize(sum([x[i] for i in range(6)]))
    
    sol = solver.Solve()
    
    print('Solution:')
#     print(sol)
    print('Number of Security Guards needed =', solver.Objective().Value())
    for i in range(6):
        print(x[i]," - ", x[i].solution_value())
    
if __name__ == '__main__':
    main()

Solution:
Number of Security Guards needed = 32.0
New Joiners for shift 0  -  3.0
New Joiners for shift 1  -  2.0
New Joiners for shift 2  -  6.0
New Joiners for shift 3  -  9.0
New Joiners for shift 4  -  2.0
New Joiners for shift 5  -  10.0


Q3. Bank Loan

In [ ]:

from ortools.linear_solver import pywraplp
def main():
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('GLOP')

    # Create the decision variables.
    h = solver.NumVar(0, solver.infinity(), 'Housing Loans')
    e = solver.NumVar(0, solver.infinity(), 'Education Loans')
    s = solver.NumVar(0, solver.infinity(), 'Senior Citizen Loans')
    p = solver.NumVar(0, solver.infinity(), 'Money in project')

    #Constraints
    #Share of Housing Loans
    solver.Add(0.75*h >= 0.25*(e+s))
    solver.Add(0.4*h <= 0.6*(e+s))

    #Share of Senior Citizen Loans
    solver.Add(2*s/3 >= (h+e)/3)

    #Capacity Constraint
    solver.Add(h+e+s+p <= 25000000)#you can also use the equality constraint

    solver.Maximize(0.085*h+0.1375*e+0.1225*s+0.09*p)

    sol = solver.Solve()
    
    print('Solution:')
#     print(sol)
    print('Banks Income =', solver.Objective().Value())
    print(h, " - ", h.solution_value())
    print(e, " - ", e.solution_value())
    print(s, " - ", s.solution_value())
    print(p, ' - ', p.solution_value())
if __name__ == '__main__':
    main()

Solution:
Banks Income = 2984375.0
Housing Loans  -  6250000.0
Education Loans  -  10416666.666666666
Senior Citizen Loans  -  8333333.333333332
Money in project  -  0.0
